In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#! pip install einops

In [3]:
import sys,os
import torch
sys.path.append(os.path.abspath('../models'))
from tinyism import tinyModel

In [4]:
posencoding=True
model=tinyModel(posEncoding=posencoding)

In [5]:
 weight="../weights/resnet18_inst.pth"
 model.load_state_dict(torch.load(weight,map_location=torch.device('cpu')))
 model.eval()

tinyModel(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [27]:
    sys.path.append(os.path.abspath('../datasets'))
    from data_loader import preprocess
    import cv2
    img=cv2.imread("../test3.png")
    img=cv2.resize(img,(512,512))
    x=preprocess((torch.from_numpy(img).float())).unsqueeze(0)

In [ ]:
## Weights conversion

In [ ]:
posencoding=True
model_curr=tinyModel(posEncoding=posencoding,deploy=False)
curr_state_dict=model_curr.state_dict()

In [ ]:
device=torch.device("cpu")
loaded_state_dict=torch.load('../weights/RepVGGplus-L2pse-train256-acc84.06.pth',map_location=device)
new_state_dict={}
matched_layers=0
unmatched_layers=0
for k in curr_state_dict.keys():
    if "backbone" in k:
        modk=k.split(".")[1:]
        modk=".".join(modk)
        #print(f"mod k is {modk}")
    else:
        modk=k
    if modk in loaded_state_dict.keys() and loaded_state_dict[modk].size()==curr_state_dict[k].size():
        new_state_dict[k]=loaded_state_dict[modk]
        matched_layers+=1
    else:
        new_state_dict[k]=curr_state_dict[k]
        unmatched_layers+=1

In [ ]:
## find any layer of same size
device=torch.device("cpu")
loaded_state_dict=torch.load('../weights/RepVGGplus-L2pse-train256-acc84.06.pth',map_location=device)
new_state_dict={}
matched_layers=0
unmatched_layers=0
for k in curr_state_dict.keys():
    findsz=curr_state_dict[k].size()
    found=False
    for kl in loaded_state_dict.keys():
        foundsz=loaded_state_dict[kl].size()
        if findsz==foundsz:
            new_state_dict[k]=loaded_state_dict[kl]
            matched_layers+=1
            found=True
    if not found:
        new_state_dict[k]=curr_state_dict[k]
        unmatched_layers+=1

In [ ]:
print(f"Matched Layers {matched_layers}")
print(f"Unmatched Layers {unmatched_layers}")

In [ ]:
#loaded_state_dict.keys()

In [ ]:
#curr_state_dict.keys()

In [ ]:
torch.save(new_state_dict, "../weights/repvgg_pretrain.pth")

In [ ]:
## Load repvggplus
import torch

In [ ]:
sys.path.append(os.path.abspath('../models'))
from repvggplus_tinysim import tinyModel

In [ ]:
posencoding=True
model_curr=tinyModel(posEncoding=posencoding,deploy=False)

In [ ]:
device=torch.device("cpu")
loaded_state_dict=torch.load('../weights/RepVGGplus-L2pse-train256-acc84.06.pth',map_location=device)

In [ ]:
model_curr.backbone

In [ ]:
model_curr.backbone.load_state_dict(loaded_state_dict,strict=False)